In [5]:
import tensorflow as tf
from transformers import TFAutoModel, TFBertTokenizer
from tensorflow_similarity.models import SimilarityModel
import tensorflow_similarity as tfsim


class TFSentenceTransformer(tf.keras.layers.Layer):
    def __init__(self, model_name_or_path, **kwargs):
        super(TFSentenceTransformer, self).__init__()
        # loads transformers model
        self.model = TFAutoModel.from_pretrained(model_name_or_path, **kwargs)

    def call(self, inputs, normalize=True):
        # runs model on inputs
        model_output = self.model(inputs)
        # Perform pooling. In this case, mean pooling.
        embeddings = self.mean_pooling(model_output, inputs["attention_mask"])
        # normalizes the embeddings if wanted
        if normalize:
            embeddings = self.normalize(embeddings)
        return embeddings

    def mean_pooling(self, model_output, attention_mask):
        # First element of model_output contains all token embeddings
        token_embeddings = model_output[0]
        input_mask_expanded = tf.cast(
            tf.broadcast_to(tf.expand_dims(attention_mask, -1),
                            tf.shape(token_embeddings)),
            tf.float32
        )
        return tf.math.reduce_sum(token_embeddings * input_mask_expanded, axis=1) / tf.clip_by_value(tf.math.reduce_sum(input_mask_expanded, axis=1), 1e-9, tf.float32.max)

    def normalize(self, embeddings):
        embeddings, _ = tf.linalg.normalize(embeddings, 2, axis=1)
        return embeddings


class E2ESentenceTransformer(tf.keras.Model):
    def __init__(self, model_name_or_path):
        super().__init__()

        self.tokenizer = TFBertTokenizer.from_pretrained(model_name_or_path)
        self.model = TFSentenceTransformer(model_name_or_path)

    def call(self, inputs):
        tokenized = self.tokenizer(
            inputs, padding="max_length", max_length=128, truncation=True)
        embedding = self.model(tokenized)
        return embedding


def create_similarity_model(model_id='sentence-transformers/all-MiniLM-L6-v2'):

    # specify input layer
    tokenizer_input = tf.keras.Input(
        shape=(), dtype='string', name="text_inputs")
    # loading model with tokenizer and sentence transformer
    e2e_model = E2ESentenceTransformer(model_id)
    # merge model with input layer
    outputs = e2e_model(tokenizer_input)

    # create similarity model
    sim_model = tfsim.models.SimilarityModel(tokenizer_input, outputs)
    return sim_model


/Users/stefruinard/Documents/personal/projects/DeepLearningCareerGuide/machine_learning_systems/src/services/search/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sim_model = create_similarity_model()



ModuleNotFoundError: No module named 'tensorflow'